In [1]:
from ib_insync import *
util.startLoop()

In [2]:
import time
from datetime import datetime, timedelta
from ib_insync import *

# Connect to TWS or IB Gateway
ib = IB()
ib.connect()  # Change port to 7496 for live account



<IB connected to 127.0.0.1:7497 clientId=1>

In [3]:

# Parameters
symbol = 'AAPL'
quantity = 10
risk_to_reward_ratio = 2

# Define the contract
contract = Stock(symbol, 'SMART', 'USD')

# Calculate the start and end time for the first minute
current_time = datetime.now()
start_time = current_time.replace(hour=9, minute=30, second=0, microsecond=0)
end_time = start_time + timedelta(minutes=1)

# Fetch the 1-minute bar for the first minute
def get_opening_range():
    while True:
        bars = ib.reqHistoricalData(
            contract,
            endDateTime=start_time,
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
        if bars:
            return bars[-1]
        time.sleep(1)

opening_range = get_opening_range()
opening_high = opening_range.high
opening_low = opening_range.low

# Function to place an order
def place_order(action, qty, contract):
    order = MarketOrder(action, qty)
    trade = ib.placeOrder(contract, order)
    return trade

# Monitor the breakout
def monitor_breakout():
    while True:
        ticker = ib.reqMktData(contract, '', False, False)
        ib.sleep(1)
        latest_price = ticker.last
        
        if latest_price > opening_high:
            # Long position
            trade = place_order('BUY', quantity, contract)
            stop_loss = opening_low
            take_profit = latest_price + (latest_price - opening_low) * risk_to_reward_ratio
            break
        
        elif latest_price < opening_low:
            # Short position
            trade = place_order('SELL', quantity, contract)
            stop_loss = opening_high
            take_profit = latest_price - (opening_high - latest_price) * risk_to_reward_ratio
            break
        
        time.sleep(1)
    
    return stop_loss, take_profit

stop_loss, take_profit = monitor_breakout()

# Function to monitor and exit the position
def manage_position(stop_loss, take_profit):
    while True:
        ticker = ib.reqMktData(contract, '', False, False)
        ib.sleep(1)
        latest_price = ticker.last
        
        if latest_price <= stop_loss:
            place_order('SELL', quantity, contract)
            break
        
        if latest_price >= take_profit:
            place_order('SELL', quantity, contract)
            break
        
        time.sleep(1)

manage_position(stop_loss, take_profit)




Error 10089, reqId 9: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')
Error 10089, reqId 10: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')
Error 10089, reqId 11: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')
Error 10089, reqId 12: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAP

KeyboardInterrupt: 

Error 10089, reqId 16: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; usfarm; euhmds; fundfarm; ushmds; secdefil.


In [9]:
# Disconnect from TWS
ib.disconnect()